In [270]:
import scraping as s
import util as u
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup

import json
import re

In [2]:
token = 'HTVFOfb4OTB04zvV4zfmtv8Oeu5cKecEHRA_aw4gi6-4KC2kaaszNQ6yIQxLGera'

## Notes
- Use song-artist string as "key" in my database
- Scrape genres separately

## Billboard

### Workflow into Genius

**Cycle through weeks**

In [586]:
top_20_base_url = 'https://www.billboard.com/charts/rap-song'
album_base_url = 'https://genius.com/albums/'

Scrape a week's top-20 lists. 
- Check if song-artist entry already in 'song_artist_list'
- if not, "do_all_the_things"

Missing entry = Quavo - WORKIN ME - https://genius.com/Quavo-workin-me-lyrics

Can't pickle dictionary "recursion limit reached". Use MongoDB instead

## MongoDB

In [329]:
from pymongo import MongoClient
from pymongo import InsertOne, DeleteOne, ReplaceOne

SSH tunnel in thru command line:

```ssh -NL 12345:localhost:27017 kbfreder@```

Also start Mongo from command line -- log onto AWS

In [374]:
client = MongoClient(port=12345)

In [396]:
client.list_database_names()

['admin', 'local', 'song_db']

In [376]:
db = client.song_db

In [377]:
db.collection_names()

['song_db']

In [383]:
# col = db.get_collection('song_db')
# col = db.create_collection('lyrics')
col = db.get_collection('lyrics')

In [585]:
col.count()

0

In [592]:
# old_song_artist_list = song_artist_list.copy()

In [593]:
# only initialize these the first time
song_artist_list = []
album_url_list = []

In [674]:
first_date = dt.datetime(2010,1,1)
stop_date = dt.datetime(2008,1,1)

next_date = first_date
temp_dict = {}

while next_date > stop_date:
    next_url = next_date.strftime("%Y-%m-%d")
    url = f'{top_20_base_url}/{next_url}'
    print(f'Scraping charts for week of: {next_url}')#, end = '\r')

    new_list = s.scrape_top_20(url)
    
    for entry in new_list:
        if entry not in song_artist_list:
#             print("Adding new entry! ", entry)
            song_entry = s.get_song_entry(entry)
            temp_dict[song_entry[0]] = song_entry[1]
            song_artist_list.append(entry)
            
            album_url = song_entry[2]
            if album_url not in album_url_list:
                album_url_list.append(album_url)
        
    if len(temp_dict) > 20:
        print("Performing an insert")
        col.insert_many(temp_dict.values())
        temp_dict = {}
        
    next_date -= dt.timedelta(days=7)


if len(temp_dict) != 0:
    col.insert_many(temp_dict.values())
    temp_dict={}

print ('Done')

Scraping charts for week of: 2010-01-01
Performing an insert
Scraping charts for week of: 2009-12-25
Scraping charts for week of: 2009-12-18
Scraping charts for week of: 2009-12-11
Scraping charts for week of: 2009-12-04
Scraping charts for week of: 2009-11-27
Scraping charts for week of: 2009-11-20
Scraping charts for week of: 2009-11-13
Scraping charts for week of: 2009-11-06
Scraping charts for week of: 2009-10-30
Scraping charts for week of: 2009-10-23
Scraping charts for week of: 2009-10-16
Scraping charts for week of: 2009-10-09
Scraping charts for week of: 2009-10-02
Performing an insert
Scraping charts for week of: 2009-09-25
Scraping charts for week of: 2009-09-18
Scraping charts for week of: 2009-09-11
Scraping charts for week of: 2009-09-04
Scraping charts for week of: 2009-08-28
Scraping charts for week of: 2009-08-21
Scraping charts for week of: 2009-08-14
Scraping charts for week of: 2009-08-07
Scraping charts for week of: 2009-07-31
Scraping charts for week of: 2009-07-2

In [636]:
len(temp_dict)

0

In [628]:
# col.insert_many(temp_dict.values())

In [675]:
col.count()

1093

In [676]:
len(song_artist_list)

1112

In [677]:
len(album_url_list)

473

In [678]:
# Check for duplicates
len(list(col.aggregate ( [
    {'$group': {'_id': {"song":"$song", "artist":"$artist"}, "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
    {"$match": {"count": { "$gte": 2}}},
    { "$group": { '_id': None, "count": { "$sum": 1 } } }
    ])))


0

## Make a back-up

In [679]:
db.create_collection('lyrics_backup')

Collection(Database(MongoClient(host=['localhost:12345'], document_class=dict, tz_aware=False, connect=True), 'song_db'), 'lyrics_backup')

In [680]:
pipeline = [ {"$match": {}}, 
             {"$out": "lyrics_backup"},
]

col.aggregate(pipeline)

In [681]:
db.lyrics_backup.estimated_document_count()

1093

### Add 'genre' field to lyrics collection

In [690]:
db.lyrics_backup.update_many({},
                          {'$set' : {"genre":"rap"}},
                          upsert = False)

In [691]:
db.lyrics_backup.find_one()

{'_id': ObjectId('5be20619a034d70ca6902a09'),
 'song': 'Lucid Dreams',
 'artist': 'Juice WRLD',
 'album': 'JuiceWRLD 9 9 9',
 'lyrics': "\n\n[Intro]\nEnviyon on the mix\nNo, no, no, no\nNo-no, no, no, no\nNo, no, no, no, no\nNo, no, no, no\n\n[Chorus]\nI still see your shadows in my room\nCan't take back the love that I gave you\nIt's to the point where I love and I hate you\nAnd I cannot change you, so I must replace you, oh\nEasier said than done, I thought you were the one\nListenin' to my heart instead of my head\nYou found another one, but I am the better one\nI won't let you forget me\nI still see your shadows in my room\nCan't take back the love that I gave you\nIt's to the point where I love and I hate you\nAnd I cannot change you, so I must replace you, oh\nEasier said than done, I thought you were the one\nListenin' to my heart instead of my head\nYou found another one, but I am the better one\nI won't let you forget me\n\n[Verse]\nYou left me falling and landing inside my gr

In [692]:
col.update_many({}, {'$set' : {"genre":"rap"}}, upsert = False)

In [693]:
col.find_one()

{'_id': ObjectId('5be20619a034d70ca6902a09'),
 'song': 'Lucid Dreams',
 'artist': 'Juice WRLD',
 'album': 'JuiceWRLD 9 9 9',
 'lyrics': "\n\n[Intro]\nEnviyon on the mix\nNo, no, no, no\nNo-no, no, no, no\nNo, no, no, no, no\nNo, no, no, no\n\n[Chorus]\nI still see your shadows in my room\nCan't take back the love that I gave you\nIt's to the point where I love and I hate you\nAnd I cannot change you, so I must replace you, oh\nEasier said than done, I thought you were the one\nListenin' to my heart instead of my head\nYou found another one, but I am the better one\nI won't let you forget me\nI still see your shadows in my room\nCan't take back the love that I gave you\nIt's to the point where I love and I hate you\nAnd I cannot change you, so I must replace you, oh\nEasier said than done, I thought you were the one\nListenin' to my heart instead of my head\nYou found another one, but I am the better one\nI won't let you forget me\n\n[Verse]\nYou left me falling and landing inside my gr

## Save stuff

- incorporate inserting / writing in MongoDB as part of work flow
- cycle over more weeks

In [1]:
# import dill
# dill.dump_session('Data/session_dumps/session_20181106.pkl')

In [694]:
u.pkl_this('Data/song_artist_list.pkl', song_artist_list)
u.pkl_this('Data/album_url_list.pkl', album_url_list)